In [3]:
import pandas as pd
from pyvis.network import Network

export_file = "/Users/kejiyuan/Desktop/export.csv"
sorted_by_ig_file = "/Users/kejiyuan/Desktop/sorted_by_ig1030.csv"

export_df = pd.read_csv(export_file)
export_df['ReactionLikeEvent'] = export_df['ReactionLikeEvent'].str.strip()
sorted_by_ig_df = pd.read_csv(sorted_by_ig_file)
sorted_by_ig_df = sorted_by_ig_df['ig_0\tig_1\tstart\tend'].str.split('\t', expand=True)
sorted_by_ig_df.columns = ['ig_0', 'ig_1', 'start', 'end']

sorted_by_ig_df['start'] = sorted_by_ig_df['start'].str.strip()
sorted_by_ig_df['end'] = sorted_by_ig_df['end'].str.strip()
sorted_by_ig_df['ig_1'] = pd.to_numeric(sorted_by_ig_df['ig_1'])

merged_df = sorted_by_ig_df.merge(
    export_df,
    left_on='start',
    right_on='ReactionLikeEvent',
    how='left'
).rename(columns={'Title': 'start_title'})

merged_df = merged_df.merge(
    export_df,
    left_on='end',
    right_on='ReactionLikeEvent',
    how='left'
).rename(columns={'Title': 'end_title'})

merged_df['start_title'] = merged_df['start_title'].fillna('')
merged_df['end_title'] = merged_df['end_title'].fillna('')

net = Network(notebook=True)

for _, row in merged_df.iterrows():
    net.add_node(row['start'], label=row['start'], title=row['start_title'])
    net.add_node(row['end'], label=row['end'], title=row['end_title'])

original_edges = [{"from": row["start"], "to": row["end"], "value": row["ig_1"]} for _, row in merged_df.iterrows()]

edges = [{"from": edge["from"], "to": edge["to"], "value": edge["value"]} for edge in original_edges if edge["value"] >= 0.5]

nodes = [{"id": node["id"], "label": node["label"], "title": node["title"]} for node in net.nodes]

html_template = f"""
<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8">
    <script type="text/javascript" src="https://unpkg.com/vis-network/standalone/umd/vis-network.min.js"></script>
    <style type="text/css">
        #mynetwork {{
            width: 100%;
            height: 800px;
            border: 1px solid lightgray;
        }}
        #slider {{
            margin-top: 10px;
        }}
    </style>
</head>
<body>
<div id="mynetwork"></div>
<div id="slider">
    <input type="range" min="0" max="1" step="0.01" value="0.5" id="weightSlider" oninput="updateNetwork()">
    <label for="weightSlider">Edge Weight Threshold: <span id="threshold">0.5</span></label>
</div>
<script type="text/javascript">
    var nodes = {nodes};
    var original_edges = {original_edges};
    var container = document.getElementById('mynetwork');
    var data = {{
        nodes: new vis.DataSet(nodes),
        edges: new vis.DataSet([])
    }};
    var options = {{
        edges: {{
            scaling: {{
                min: 1,
                max: 15,
                label: {{
                    enabled: true
                }},
                min: 0.5
            }}
        }},
        physics: {{
            enabled: true,
            barnesHut: {{
                gravitationalConstant: -2000,
                centralGravity: 0.3,
                springLength: 95,
                springConstant: 0.04,
                damping: 0.09
            }},
            solver: 'barnesHut'
        }},
    }};
    var network = new vis.Network(container, data, options);

    function updateNetwork() {{
        var threshold = parseFloat(document.getElementById('weightSlider').value);
        document.getElementById('threshold').innerText = threshold;
        var filtered_edges = original_edges.filter(edge => edge.value >= threshold);
        var connected_nodes = new Set();
        filtered_edges.forEach(edge => {{
            connected_nodes.add(edge.from);
            connected_nodes.add(edge.to);
        }});
        var filtered_nodes = nodes.filter(node => connected_nodes.has(node.id));
        data.nodes.clear();
        data.nodes.add(filtered_nodes);
        data.edges.clear();
        data.edges.add(filtered_edges);
    }}

    updateNetwork();
</script>
</body>
</html>
"""

with open("reactions_graph.html", "w") as f:
    f.write(html_template)

from IPython.display import IFrame
IFrame("reactions_graph.html", width=800, height=800)


In [43]:
import pandas as pd
from pyvis.network import Network
from IPython.display import IFrame

export_file = "/Users/kejiyuan/Desktop/export.csv"
sorted_by_ig_file = "/Users/kejiyuan/Desktop/sorted_by_ig1030.csv"

export_df = pd.read_csv(export_file)
export_df['ReactionLikeEvent'] = export_df['ReactionLikeEvent'].str.strip()
sorted_by_ig_df = pd.read_csv(sorted_by_ig_file)
sorted_by_ig_df = sorted_by_ig_df['ig_0\tig_1\tstart\tend'].str.split('\t', expand=True)
sorted_by_ig_df.columns = ['ig_0', 'ig_1', 'start', 'end']
sorted_by_ig_df = sorted_by_ig_df.drop_duplicates()

sorted_by_ig_df['start'] = sorted_by_ig_df['start'].str.strip()
sorted_by_ig_df['end'] = sorted_by_ig_df['end'].str.strip()
sorted_by_ig_df['ig_1'] = pd.to_numeric(sorted_by_ig_df['ig_1'])

merged_df = sorted_by_ig_df.merge(
    export_df,
    left_on='start',
    right_on='ReactionLikeEvent',
    how='left'
).rename(columns={'Title': 'start_title'})

merged_df = merged_df.merge(
    export_df,
    left_on='end',
    right_on='ReactionLikeEvent',
    how='left'
).rename(columns={'Title': 'end_title'})

merged_df['start_title'] = merged_df['start_title'].fillna('')
merged_df['end_title'] = merged_df['end_title'].fillna('')

net = Network(notebook=True)

for _, row in merged_df.iterrows():
    net.add_node(row['start'], label=row['start'], title=row['start_title'])
    net.add_node(row['end'], label=row['end'], title=row['end_title'])

original_edges = [{"from": row["start"], "to": row["end"], "value": row["ig_1"]} for _, row in merged_df.iterrows()]

nodes = [{"id": node["id"], "label": node["label"], "title": node["title"]} for node in net.nodes]

html_template = f"""
<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8">
    <script type="text/javascript" src="https://unpkg.com/vis-network/standalone/umd/vis-network.min.js"></script>
    <style type="text/css">
        #mynetwork {{
            width: 100%;
            height: 800px;
            border: 1px solid lightgray;
        }}
        #slider {{
            margin-top: 10px;
        }}
    </style>
</head>
<body>
<div id="mynetwork"></div>
<div id="slider">
    <input type="range" min="0" max="1" step="0.01" value="0.5" id="weightSlider" oninput="updateNetwork()">
    <label for="weightSlider">Edge Weight Threshold: <span id="threshold">0.5</span></label>
</div>
<script type="text/javascript">
    var nodes = {nodes};
    var original_edges = {original_edges};
    var container = document.getElementById('mynetwork');
    var data = {{
        nodes: new vis.DataSet(nodes),
        edges: new vis.DataSet([])
    }};
    var options = {{
        nodes: {{
        shape: 'box',
        size: 20,
        font: {{ size: 14, color: '#ffffff' }},
        margin: 10
        }},
        edges: {{
            arrows: {{
                to: {{ enabled: true, scaleFactor: 0.5, }}  
            }},
            width: 1,  
            color: {{
                inherit: false
            }}
        }},
        physics: {{
            enabled: true,
            barnesHut: {{
                gravitationalConstant: -2000,
                centralGravity: 0.3,
                springLength: 300,
                springConstant: 0.04,
                damping: 0.09
            }},
            solver: 'barnesHut'
        }},
    }};

    var network = new vis.Network(container, data, options);

    function updateNetwork() {{
        var threshold = parseFloat(document.getElementById('weightSlider').value);
        document.getElementById('threshold').innerText = threshold;
        var filtered_edges = original_edges.filter(edge => edge.value >= threshold).map(edge => {{
            edge.color = `hsl(${{30 - (edge.value - 0.5) * 240}}, 100%, 50%)`;  // Color from yellow (low weight) to dark purple (high weight)
            return edge;
        }});
        var connected_nodes = new Set();
        filtered_edges.forEach(edge => {{
            connected_nodes.add(edge.from);
            connected_nodes.add(edge.to);
        }});
        var filtered_nodes = nodes.filter(node => connected_nodes.has(node.id));
        data.nodes.clear();
        data.nodes.add(filtered_nodes);
        data.edges.clear();
        data.edges.add(filtered_edges);
    }}

    updateNetwork();
</script>
</body>
</html>
"""

# Write the HTML output to a file and display it
with open("reactions_graph.html", "w") as f:
    f.write(html_template)

IFrame("reactions_graph.html", width=960, height=800)
